## QA on wikipedia with Dense Passage Retrieval

#### document_store

In [1]:
HOST = 'localhost' 
PORT = 9200 
INDEX_NAME = 'wikipedia_en'

import logging
logging.disable(logging.INFO)
logging.disable(logging.WARNING)
from haystack import Finder
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host=HOST, port=PORT, username="", password="", index=INDEX_NAME)

#### retriever (ES)

In [2]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever_es = ElasticsearchRetriever(document_store=document_store)

In [4]:
docs = retriever_es.retrieve('who is obama?', top_k=10)

In [15]:
docs[0].query_score

14.246899

#### retriever (dense)

In [3]:
from haystack.retriever.dense import DensePassageRetriever
retriever_dense = DensePassageRetriever(document_store=document_store,
                                  embedding_model='../models/dpr/multi_hf_bert_base.cp',
                                  use_gpu=True,
                                  batch_size=16,
                                  do_lower_case=True)

In [4]:
q_vec = retriever_dense.embed_queries(['who is obama?'])

In [5]:
q_vec[0].shape

(768,)

#### reader (electra-base-squad2)

In [9]:
READER_DiR = "../models/electra-base-squad2"
reader = TransformersReader(model=READER_DiR, tokenizer=READER_DiR,  use_gpu=0)

#### finder

In [ ]:
finder = Finder(reader, retriever)
prediction = finder.get_answers(question="who is the father of Arya Stark", 
                                top_k_retriever=10, 
                                top_k_reader=3)